In [2]:
from argparse import ArgumentParser
from json import dump
from logging import basicConfig, getLogger
from os import linesep, remove
from os.path import exists
from tempfile import NamedTemporaryFile
from typing import Dict, List, Tuple

from requests import get
from sentencepiece import SentencePieceProcessor
from tqdm import trange, tqdm

In [3]:
basicConfig()

In [4]:
# class SentencePieceExtractor:
#     """
#     Extractor implementation for SentencePiece trained models.
#     https://github.com/google/sentencepiece
#     """

#     def __init__(self, model: str):
#         # Get SentencePiece
#         self.sp = SentencePieceProcessor()
#         self.sp.Load(model)

#     def extract(self) -> Tuple[Dict[str, int], List[Tuple]]:
#         sp = self.sp
#         vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())}

#         # Merges
#         merges = []
#         for piece_l in tqdm(vocab.keys(), total=sp.GetPieceSize()):
#             for piece_r in vocab.keys():
#                 merge = f"{piece_l}{piece_r}"
#                 piece_id = vocab.get(merge, None)
#                 if piece_id:
#                     merges += [(piece_l, piece_r, piece_id)]
#         merges = sorted(merges, key=lambda val: val[2])
#         merges = [(val[0], val[1]) for val in merges]

#         return vocab, merges

In [5]:
# from itertools import chain
# from multiprocessing import Pool, cpu_count

# class SentencePieceExtractor:
#     """
#     Extractor implementation for SentencePiece trained models.
#     https://github.com/google/sentencepiece
#     """

#     def __init__(self, model: str):
#         # Get SentencePiece
#         self.sp = SentencePieceProcessor()
#         self.sp.Load(model)
    
#     def extract(self) -> Tuple[Dict[str, int], List[Tuple]]:
#         sp = self.sp
#         vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())} 

#         results = []
#         with Pool(cpu_count()) as pool:
#             results = pool.starmap(self.extract_merges, [(key, vocab,) for key in vocab.keys()])

#         # Flatten and filter empty lists
#         merges = list(chain.from_iterable(filter(None, results)))

#         merges.sort(key=lambda val: val[2])
#         merges = [(val[0], val[1]) for val in merges]
        
#         return vocab, merges

#     """
#     Multiprocessing for merges.
#     """
#     @staticmethod
#     def extract_merges(piece_l, vocab):
#         merges = []
#         for piece_r in vocab.keys():
#             merge = f"{piece_l}{piece_r}"
#             piece_id = vocab.get(merge, None)
#             if piece_id:
#                 merges += [(piece_l, piece_r, piece_id)]
        
#         return merges

In [7]:
sp = SentencePieceProcessor()
sp.Load("/DATA/rohit/NLP_2025/unigram_mixed.model")

True

In [8]:
vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())}

100%|██████████| 3200/3200 [00:00<00:00, 350035.80it/s]


In [9]:
from multiprocessing import Pool, cpu_count

In [10]:
def process_merges(piece_l, vocab):    
    merges = []
    for piece_r in vocab.keys():
        merge = f"{piece_l}{piece_r}"
        piece_id = vocab.get(merge, None)
        if piece_id:
            merges += [(piece_l, piece_r, piece_id)]
    
    return merges

In [11]:
## Think about sorting later, aim should be correctness

In [12]:
results = []
with Pool(cpu_count()) as pool:
    results = pool.starmap(process_merges, [(k, vocab,) for k in vocab.keys()])

In [13]:
merges = []
for i in results:
    if i != []:
        merges += i

In [14]:
merges = sorted(merges, key=lambda val: val[2])
merges = [(val[0], val[1]) for val in merges]

In [15]:
import json

In [16]:
with open("vocab_fast.json", "w") as f:
    json.dump(vocab, f)

In [17]:
type(merges)

list

In [18]:
merges[0]

('▁', 'के')

In [19]:
with open("merges_fast.txt", "w") as f:
    for item in merges:
        f.write(f"{item[0]} {item[1]}\n")

In [20]:
from tokenizers import SentencePieceBPETokenizer

In [21]:
tokenizer = SentencePieceBPETokenizer()

In [22]:
test = tokenizer.from_file("vocab_fast.json", "merges_fast.txt")

In [23]:
test

Tokenizer(vocabulary_size=3200, model=SentencePieceBPE, unk_token=<unk>, replacement=▁, add_prefix_space=True, dropout=None)

In [24]:
test.save_model("/DATA/rohit/NLP_2025/uni_pretok_explicit")

['/DATA/rohit/NLP_2025/uni_pretok_explicit/vocab.json',
 '/DATA/rohit/NLP_2025/uni_pretok_explicit/merges.txt']

In [25]:
test.save("/DATA/rohit/NLP_2025/uni_pretok_explicit/tokenizer.json")

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="Final/tokenizer.json")

In [ ]:
tokenizer.save_pretrained("Paka_Final")

('Paka_Final/tokenizer_config.json',
 'Paka_Final/special_tokens_map.json',
 'Paka_Final/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Paka_Final", use_fast=True)

In [ ]:
tokenizer.encode("How are tyou sff <BOS>ffds dsa<EOS>1!)")

[13576,
 6047,
 11980,
 5304,
 5315,
 5522,
 124378,
 3,
 5318,
 83741,
 5348,
 11795,
 4,
 124378,
 124400,
 64684]